### Imports

In [1]:
import os
import sys
sys.path.insert(0, '../')
import tensorflow as tf
from tensorflow.keras.models import load_model

import torch
from torch.utils.data import DataLoader, TensorDataset
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.utils.metrics import IoU, Precision, Recall
from segmentation_models_pytorch.utils.losses import DiceLoss

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage import util
from pathlib import Path
from skimage import util, filters, morphology

from preprocessing.preprocess_data import get_preprocessed_data, TRAIN_PATH, _read_image
from mask_to_submission import *


2024-07-31 12:12:18.708110: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-31 12:12:19.230710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 12:12:19.380899: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 12:12:19.431073: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-31 12:12:19.742198: I tensorflow/core/platform/cpu_feature_guar

ModuleNotFoundError: No module named 'segmentation_models_pytorch'

### Loading the dataset

In [2]:
val_size = 0.2

(X_train, Y_train), (X_val, Y_val) = get_preprocessed_data(path=TRAIN_PATH, val_size=val_size)

print(f'Training data shapes: X_train: {X_train.shape}, Y_train: {Y_train.shape}')
print(f'Validation data shapes: X_val: {X_val.shape}, Y_val: {Y_val.shape}')

NameError: name 'get_preprocessed_data' is not defined

#### Training

In [5]:
model = smp.UnetPlusPlus(
        encoder_name="resnet34",
        encoder_weights='imagenet',
        classes=1, 
        activation='sigmoid'  
    )

In [8]:
# Define the padding function
def pad_to_32(image):
    h, w = image.shape[:2]
    new_h, new_w = (h + 31) // 32 * 32, (w + 31) // 32 * 32
    pad_h, pad_w = new_h - h, new_w - w
    image = np.pad(image, ((0, pad_h), (0, pad_w), (0, 0)), mode='constant', constant_values=0)
    return image

# Apply padding to the dataset
def pad_images(images):
    padded_images = [pad_to_32(img) for img in images]
    return padded_images

# Pad the training and validation images
X_train_padded = pad_images(X_train)
Y_train_padded = pad_images(Y_train)
X_val_padded = pad_images(X_val)
Y_val_padded = pad_images(Y_val)

In [9]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_padded, dtype=torch.float32).permute(0, 3, 1, 2)
Y_train_tensor = torch.tensor(Y_train_padded, dtype=torch.float32).permute(0, 3, 1, 2)
X_val_tensor = torch.tensor(X_val_padded, dtype=torch.float32).permute(0, 3, 1, 2)
Y_val_tensor = torch.tensor(Y_val_padded, dtype=torch.float32).permute(0, 3, 1, 2)

# Create TensorDataset and DataLoader
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


/var/folders/tg/k5r1c31d7v9g56r0919lqbz40000gn/T/ipykernel_3478/2500452614.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  X_train_tensor = torch.tensor(X_train_padded, dtype=torch.float32).permute(0, 3, 1, 2)


In [ ]:
EPOCHS = 30
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss = DiceLoss()

metrics = [
    IoU(threshold=0.5),
    Precision(threshold=0.5),
    Recall(threshold=0.5),
    Fscore(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.00008),
])

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, T_0=1, T_mult=2, eta_min=5e-5,
)

train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

best_iou_score = 0.0
train_logs_list, valid_logs_list = [], []
for i in range(0, EPOCHS):
    # Perform training & validation
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(val_loader)
    train_logs_list.append(train_logs)
    valid_logs_list.append(valid_logs)
    # Save model if a better val IoU score is obtained
    if best_iou_score < valid_logs['iou_score']:
        best_iou_score = valid_logs['iou_score']
        torch.save(model, './modelu++.pth')
        print('Model saved!')


Epoch: 0
valid: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [03:15<00:00, 11.50s/it, dice_loss - 0.5292, iou_score - 0.4164, precision - 0.4488, recall - 0.8535]
Model saved!

Epoch: 1
valid: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [03:11<00:00, 11.28s/it, dice_loss - 0.4681, iou_score - 0.486, precision - 0.5874, recall - 0.7416]
Model saved!

Epoch: 2
valid: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [03:12<00:00, 11.33s/it, dice_loss - 0.4345, iou_score - 0.5013, precision - 0.5806, recall - 0.7876]
Model saved!

Epoch: 3
valid: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [03:11<00:00, 11.24s/it, dice_loss - 0.4072, iou_score - 0.5175, precision - 0.6185, recall - 0.7623]
Model saved!

Epoch: 4
valid: 100%|██████████████████████████

### Predict

In [ ]:
# Function to visualize the predictions
def visualize_predictions(model, val_loader, device, num_images=5):
    """Visualizes the predictions made by a given model on a validation dataset.

    Args:
        model (torch.nn.Module): The model to visualize predictions for.
        val_loader (torch.utils.data.DataLoader): The validation data loader.
        device (torch.device): The device to run the model on.
        num_images (int, optional): The number of images to visualize. Defaults to 5.
    """
    model.eval()  # Set model to evaluation mode
    fig, axes = plt.subplots(num_images, 3, figsize=(15, num_images * 5))
    
    with torch.no_grad():  # Disable gradient computation
        for i, (images, masks) in enumerate(val_loader):
            images = images.to(device)
            print(images.shape)
            masks = masks.to(device)
            predictions = model(images)
            
            for j in range(min(num_images, images.size(0))):
                image = images[j].cpu().permute(1, 2, 0).numpy()
                print(image.shape)

                mask = masks[j].cpu().permute(1, 2, 0).numpy()
                prediction = predictions[j].cpu().permute(1, 2, 0).numpy()
                
                # Plot original image
                axes[j, 0].imshow(image)
                axes[j, 0].set_title("Original Image")
                axes[j, 0].axis('off')
                
                # Plot ground truth mask
                axes[j, 1].imshow(mask, cmap='gray')
                axes[j, 1].set_title("Ground Truth Mask")
                axes[j, 1].axis('off')
                
                # Plot predicted mask
                axes[j, 2].imshow(prediction, cmap='gray')
                axes[j, 2].set_title("Predicted Mask")
                axes[j, 2].axis('off')
            
            if i + 1 >= num_images // images.size(0):
                break

# Visualize the predictions
best_model = torch.load('./best_model.pth', map_location=DEVICE)
visualize_predictions(best_model, val_loader, DEVICE, num_images=5)

In [ ]:
# if you dont have memory issues, you can not predict in batches
def get_preprocessed_test_data(path: str):
    """Load and preprocess the test data.

    Args:
        path (str): The path to the test data.

    Returns:
        np.ndarray: The preprocessed test data.
    """
    test_x_files = sorted([filename for filename in os.listdir(path) if filename.endswith('.png')])
    test_x = np.array([_read_image(f'{path}/{filename}') for filename in test_x_files])
    return test_x, test_x_files

# Assuming DiceLoss is defined elsewhere
dice_loss = DiceLoss()

model = best_model
model = model.to(DEVICE)
model.eval()

X_test, X_test_files = get_preprocessed_test_data(path='ethz-cil-road-segmentation-2024/test/images')

X_test = np.array(X_test)
X_test = pad_images(X_test)


X_test = np.transpose(X_test, (0, 3, 1, 2))
X_test = torch.from_numpy(X_test)
X_test = X_test.to(DEVICE)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs for parallel processing")
    model = torch.nn.DataParallel(model)
print("predicting")
# Make predictions
y_pred = model(X_test)

# Ensure the predictions are of shape (height, width) before converting to images
y_pred = (y_pred > 0.5).astype(np.uint8).squeeze(axis=-1)  # Squeeze the last dimension if necessary

pred_images = [Image.fromarray((y_pred[i] * 255)).resize((400, 400)) for i in range(y_pred.shape[0])]

output_path = 'outputs2'
os.makedirs(output_path, exist_ok=True)  # Ensure the output directory exists

# Save all images in the output_path directory
for i, img in enumerate(pred_images):
    img.save(f'{output_path}/pred_{X_test_files[i]}')

print("Images have been successfully saved.")

/Users/eliotullmo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/eliotullmo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


predicting


### Post processing

In [ ]:
def th_li(image):
    """Apply Li thresholding."""
    threshold_value = filters.threshold_li(image)
    binary_image = image > threshold_value
    return binary_image.astype(np.uint8)

def th_manual(image, manual_t):
    """Apply manual thresholding."""
    binary_image = image > (manual_t * 255)
    return binary_image.astype(np.uint8)

def morph_closing(image, filter_size=20):
    """Apply morphological closing."""
    selem = morphology.disk(filter_size)
    closed_image = morphology.closing(image, selem)
    return closed_image

def morph_area_opening(image, area_size=400):
    """Apply morphological area opening."""
    opened_image = morphology.remove_small_objects(image.astype(bool), min_size=area_size)
    return opened_image.astype(np.uint8)

In [ ]:
def postprocess_images(pred_images, filenames, output_path):
    Path(output_path).mkdir(parents=True, exist_ok=True)
    manual_t = 0.3

    final_images = []
    for i, im in enumerate(pred_images):
        # Apply thresholding
        im_li = th_li(im)
        im_manual = th_manual(im, manual_t)

        # Apply morphological operations
        first = morph_closing(im_li, filter_size=6)
        new_im = morph_area_opening(first, area_size=400)

        new_im = np.expand_dims(new_im, axis=2)
        name = output_path + "pred_satimage_" + str(filenames[i][14:17]) + ".png"
        cv2.imwrite(name, new_im * 255)  # Ensure image is saved correctly
        final_images.append(new_im)
        print(f"Written to {name}")
    return final_images

In [ ]:
def load_predicted_images(path: str):
    """Load the predicted images from the specified path."""
    filenames = sorted([f for f in os.listdir(path) if f.endswith('.png')])
    images = [cv2.imread(os.path.join(path, f), cv2.IMREAD_GRAYSCALE) for f in filenames]
    return images, filenames

predictions_path = 'insert'
output_path = 'insert'

pred_images, filenames = load_predicted_images(predictions_path)
pred_images = [util.img_as_ubyte(im) for im in pred_images]

postprocess_images(pred_images, filenames, output_path)